In [1]:
## Pull in OS for directory handling and WRDS for data requests
## We use 'pprint' to look at outputs, which is a pretty print function for dictionaries

import os
import wrds
import pprint
import sqlite3
import multiprocessing
import pandas as pd
from concurrent.futures import ProcessPoolExecutor

In [2]:
### Initialize the connection. Unless you have SSH setup for wrds you will be prompted for a username and password each time you run this. As PUTTY is annoying and this task doesn't take long for most uses, just enter it each time.

##

# After entering the username and password, it might sit to authenticate for awhile.
# This is because it is sending me a 2-Factor Authentication request. This happens one a month.
# So just text me before you run this and I will hit the approve button on the 2FA app. 

db = wrds.Connection()

## When prompted to create a pgpass file, just type "n". As we are not using PUTTY, this will do nothing, not to worry.

WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [3]:
# Define Library and Table
library = 'ciq_transcripts'
table = "ciqtranscriptcomponent"

In [4]:
### List out the libraries SMU subscribes to:

# We are looking for transcripts provided by Capital IQ, so we will use ciq_transcripts

db.list_libraries()

['aha_sample',
 'ahasamp',
 'auditsmp',
 'auditsmp_all',
 'bank',
 'bank_all',
 'bank_premium_samp',
 'banksamp',
 'block',
 'block_all',
 'boardex_trial',
 'boardsmp',
 'bvd_amadeus_trial',
 'bvd_bvdbankf_trial',
 'bvd_orbis_trial',
 'bvdsamp',
 'calcbench_trial',
 'calcbnch',
 'cboe',
 'cboe_all',
 'cboe_sample',
 'cboesamp',
 'ciq',
 'ciq_common',
 'ciq_transcripts',
 'ciqsamp',
 'ciqsamp_capstrct',
 'ciqsamp_common',
 'ciqsamp_keydev',
 'ciqsamp_pplintel',
 'ciqsamp_ratings',
 'ciqsamp_transactions',
 'ciqsamp_transcripts',
 'cisdmsmp',
 'columnar',
 'comp',
 'comp_bank',
 'comp_bank_daily',
 'comp_global',
 'comp_global_daily',
 'comp_na_annual_all',
 'comp_na_daily_all',
 'comp_na_monthly_all',
 'comp_segments_hist',
 'comp_segments_hist_daily',
 'compa',
 'compb',
 'compg',
 'compm',
 'compsamp',
 'compsamp_all',
 'compsamp_snapshot',
 'compseg',
 'contrib',
 'contrib_as_filed_financials',
 'contrib_char_returns',
 'contrib_corporate_culture',
 'contrib_general',
 'contrib_globa

In [5]:
tables_available_from_library = db.list_tables(library)

In [6]:
### Run the following to print the tables

pprint.pprint(tables_available_from_library)

['ciqtranscript',
 'ciqtranscriptcollectiontype',
 'ciqtranscriptcomponent',
 'ciqtranscriptcomponenttype',
 'ciqtranscriptdelayreason',
 'ciqtranscriptdelayreasontype',
 'ciqtranscriptperson',
 'ciqtranscriptpresentationtype',
 'ciqtranscriptspeakertype',
 'wrds_transcript_detail',
 'wrds_transcript_person']


In [ ]:
#### The following loop will print all the tables available in the pre-pecified library, and their number of rows. 

for table in tables_available_from_library:
    row_count = db.get_row_count(library=library,table=table)
    print(f"{table} has {row_count:,} rows")

In [8]:
### To see what the various tables look like, run the following, then inspect them with the Jupyter variable viewer in VS Code. 


for table in tables_available_from_library:
    globals()[table] = db.get_table(library=library,table=table,obs=100)

In [ ]:
### If the tables are not huge, you can run the following. However, if they more than a couple million observations each you will need a lot of RAM
### For very large tables, see the next cell

output_directory = 'D:\Data\Sustainalytics'  ### Put your output directory here - I recommend naming it something similar to the WRDS library name for later referencing. 
os.chdir(output_directory)

for table in tables_available_from_library:
    print(f"Downloading {table}")
    downloaded_table = db.get_table(library=library,table=table)
    print(f"Finished downloading {table}, saving to csv...")
    downloaded_table.to_csv(f"{table}.csv",header=True,index=False)
    print(f"Successfully saved {table} to CSV. Moving to next table.")

In [ ]:
# This will simply tell you the directory you are working in so you can find the files you just downloaded.
print(os.getcwd())

In [ ]:
import os
import time
import csv
from wrds import Connection
from concurrent.futures import ThreadPoolExecutor

# Function to establish a WRDS connection
def connect_to_wrds():
    try:
        return Connection()
    except Exception as e:
        print(f"Error connecting to WRDS: {e}")
        time.sleep(10)  # Retry after 10 seconds
        return connect_to_wrds()

# Specify your output directory
output_directory = 'D:/wrdsTables/ciqtranscriptcomponent_chunks'
os.makedirs(output_directory, exist_ok=True)  # Ensure the directory exists

# Define library and table
library = 'ciq_transcripts'  # Replace with the correct library name
table = 'ciqtranscriptcomponent'  # Replace with the correct table name

# WRDS connection
global db
db = connect_to_wrds()

# Fetch distinct `transcriptid` values, excluding NULLs
def fetch_transcript_ids():
    print("Fetching unique transcript IDs...")
    transcript_ids_query = f"""
        SELECT DISTINCT transcriptid
        FROM {library}.{table}
        WHERE transcriptid IS NOT NULL
    """
    try:
        transcript_ids_result = db.raw_sql(transcript_ids_query)
        transcript_ids = [row.transcriptid for row in transcript_ids_result.itertuples(index=False) if row.transcriptid is not None]
        print(f"Found {len(transcript_ids)} unique transcript IDs.")
        return transcript_ids  # Return all transcript IDs
    except Exception as e:
        print(f"Error fetching transcript IDs: {e}")
        return []

# Save data chunk to CSV
def save_to_csv(chunk, output_file):
    with open(output_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Write column headers
        writer.writerow(chunk.columns)
        # Write rows of data
        writer.writerows(chunk.values)

# Process a single transcript ID
def process_transcript(transcript_id, index, total_ids):
    global db
    output_file = os.path.join(output_directory, f"{int(transcript_id)}.csv")

    # Skip if file already exists
    if os.path.exists(output_file):
        print(f"[{index}/{total_ids}] File {output_file} already exists. Skipping transcriptid {transcript_id}.")
        return

    print(f"[{index}/{total_ids}] Downloading data for transcriptid {transcript_id} from table: {table}")

    retries = 3
    while retries > 0:
        try:
            # Construct SQL query to fetch data for the specific transcript ID
            sql_query = f"""
                SELECT *
                FROM {library}.{table}
                WHERE transcriptid = {int(transcript_id)}
                ORDER BY componentorder ASC
            """

            # Fetch data
            chunk = db.raw_sql(sql_query)

            # Check if data is empty
            if chunk.empty:
                print(f"[{index}/{total_ids}] No data found for transcriptid {transcript_id}. Skipping...")
                return

            # Save the data to a CSV file
            save_to_csv(chunk, output_file)
            print(f"[{index}/{total_ids}] Successfully saved data for transcriptid {transcript_id} to {output_file}.")
            return

        except Exception as e:
            print(f"[{index}/{total_ids}] Error processing transcriptid {transcript_id}: {e}")
            retries -= 1
            print(f"[{index}/{total_ids}] Retrying... {retries} attempts left.")
            if retries == 0:
                print(f"[{index}/{total_ids}] Failed to process transcriptid {transcript_id} after multiple attempts. Skipping...")
            # Attempt to reconnect to the database if the connection was lost
            db = connect_to_wrds()

# Main loop for downloading data with threading
def download_transcripts_parallel(transcript_ids):
    total_ids = len(transcript_ids)
    with ThreadPoolExecutor(max_workers=4) as executor:
        executor.map(
            lambda args: process_transcript(*args),
            [(transcript_id, i + 1, total_ids) for i, transcript_id in enumerate(transcript_ids)],
        )

# Fetch and process transcript data
transcript_ids = fetch_transcript_ids()
download_transcripts_parallel(transcript_ids)

In [17]:
### If a cell errors out, and when you try to rerun the query, you get a "cannot proceed until last invalid transaction is rolled back"
### Or similar error, run this cell.

transactions_roller = db.connection.rollback()
transactions_roller

In [ ]:
# Code to delete a large folder -- irreversible
import shutil

shutil.rmtree("your file path", ignore_errors=True)  # Replace with the actual folder path